In [1]:
#Get triplets from file watdiv.10M.nt and create a table for each property
def get_triplets():
    tables = {}
    with open("watdiv.10M.nt", "r") as f:
        count = 0
        for line in f:
            object, property, subject = line.split("\t")
            object = object.split(">")[0].split("/")[-1]
            subject = subject.split(">")[0].split("/")[-1]
            property = property.split(">")[0].split("/")[-1]
            if "#" in property:
                property = property.split("#")[-1]
            
            if property not in tables:
                tables[property] = []
            tables[property].append({"object": object, "subject": subject.strip()})
    return tables

tables = get_triplets()

In [ ]:
#Get triplets from file 100k.txt and create a table for each property
def get_triplets():
    tables = {}
    with open("100k.txt", "r") as f:
        count = 0
        for line in f:
            object, property, subject = line.split("\t")
            property = property.split(":")[1]
            object = object.split(":")[-1]
            subject = subject.split(":")[-1].split(".\n")[0].strip()
            if property not in tables:
                tables[property] = []
            tables[property].append({"object": object, "subject": subject.strip()})
            # tables[property].append((object, subject.strip()))
    return tables



tables = get_triplets()

In [2]:
stringToIndex = {"object": 0, "subject": 1}
from prettytable import PrettyTable


def show(res, numRows=100000):
    print(len(res))
    count = 0
    for row in res:
        print(row)
        count += 1
        if count == numRows:
            break
def wirteToFileAsTable(res, output="output.txt"):
    print("Writing to file...")
    t = PrettyTable()
    t.field_names = ["follows.subject", "follows.object", "friendOf.object", "likes.object", "hasReview.object"]
    for row in res:
        history = list(row["history"])
        for h in history:
            if h[2] == "follows":
                followsS = h[stringToIndex["subject"]]
                followsO = h[stringToIndex["object"]]
            elif h[2] == "friendOf":
                friendOf = h[stringToIndex["object"]]
            elif h[2] == "likes":
                likes = h[stringToIndex["object"]]
        hasReview = row["subject"]
        t.add_row([followsS, followsO, friendOf, likes, hasReview])
    with open(output, "w") as f:
        f.write(t.get_string())

def wirteToFile(res, output="output.txt"):
    print("Writing to file...")
    with open(output, "w") as f:
        f.write("\t".join(["follows.subject", "follows.object", "friendOf.object", "likes.object", "hasReview.subject"]) + "\n")
        count = 0
        for row in res:
            history = list(row["history"])
            for h in history:
                if h[2] == "follows":
                    followsS = h[stringToIndex["subject"]]
                    followsO = h[stringToIndex["object"]]
                elif h[2] == "friendOf":
                    friendOf = h[stringToIndex["object"]]
                elif h[2] == "likes":
                    likes = h[stringToIndex["object"]]
            hasReview = row["subject"]
            f.write("\t".join([followsS, followsO, friendOf, likes, hasReview]) + "\n")
            count += 1
    print(count)

In [ ]:
def hash_join(table1, join_key1, table2, join_key2, historyName):
    def generate_matches(table, join_key):
        hash_table = {}
        for item in table:
            object_key = item[join_key]
            if object_key not in hash_table:
                hash_table[object_key] = []
            hash_table[object_key].append(item)

        # Use a set to store already matched keys from table2
        matched_keys = set()

        for item in table2:
            join_key_value = item[join_key2]
            if join_key_value in hash_table:
                for matched_item in hash_table[join_key_value]:
                    joined_item = {**matched_item, **item}
                    

                    # Check if the key pair has been matched before adding it to the result
                    key_pair = (joined_item[join_key1], joined_item[join_key2])
                    if key_pair not in matched_keys:
                        matched_keys.add(key_pair)
                        if "history" not in joined_item:
                            joined_item["history"] = set()

                        joined_item["history"].add((matched_item["object"], matched_item["subject"], historyName))
                        yield joined_item

    # Perform the hash join using table1 and join_key1
    joined_table = list(generate_matches(table1, join_key1))

    return joined_table

from time import time
start_time = time()

res = hash_join(tables["follows"], "object", tables["friendOf"], "subject", historyName = "follows")
# show(res, 20)

res = hash_join(res, "subject", tables["likes"], "object", historyName = "friendOf")
# show(res, 20)

res = hash_join(res, "subject", tables["hasReview"], "object", historyName = "likes")
# show(res, 20)

print("Program execution time is:", str(time()-start_time))

# wirteToFile(res, "output-hashJoin.txt")
# wirteToFileAsTable(res, "output-hashJoin-10M-table.txt")
wirteToFileAsTable(res, "output-hashJoin-100K-table.txt")

In [3]:
def sortMergeJoin(table1, index1, table2, index2, historyName):
    def merge(s, r):
        i, j = 0, 0
        while i < len(s) and j < len(r):

            key1 = s[i][index1]
            key2 = r[j][index2]

            if key1 == key2:
                joined_item = {**s[i], **r[j]}

                if "history" not in joined_item:
                        joined_item["history"] = set()
                joined_item["history"].add((s[i]["object"], s[i]["subject"], historyName))
                
                yield joined_item
                j += 1
            elif key1 < key2:
                i += 1
            else:
                j += 1
        return 

    # Sort both tables based on the join columns
    left_table = sorted(table1, key=lambda x: x[index1])
    right_table = sorted(table2, key=lambda x: x[index2])

    return list(merge(left_table, right_table))

from time import time
start_time = time()

res = sortMergeJoin(tables["follows"], "object", tables["friendOf"], "subject", "follows")
# show(res, 20)

res = sortMergeJoin(res, "subject", tables["likes"], "object", "friendOf")
# show(res, 20)

res = sortMergeJoin(res, "subject", tables["hasReview"], "object", "likes")
# show(res, 20)

print("Program execution time is:", str(time()-start_time))

# wirteToFileAsTable(res, output="output-sortMergeJoin-10M-table.txt")
wirteToFileAsTable(res, output="output-sortMergeJoin-100K-table.txt")

Program execution time is: 24.373465538024902
Writing to file...


In [ ]:
#Get triplets from file 100k.txt and create a table for each property
# and create a dictionary of index/ string
str_to_id = {}
def get_triplets():
    tables = {}
    
    with open("100k.txt", "r") as f:
        count = 0
        for line in f:
            object, property, subject = line.split("\t")
            property = property.split(":")[1]
            object = object.split(":")[-1]
            subject = subject.split(":")[-1].split(".\n")[0].strip()
            subject = subject.strip()

            if object not in str_to_id:
                str_to_id[object] = count
                obj_id = count
                count += 1
            else:
                obj_id = str_to_id[object]
                
            if subject not in str_to_id:
                str_to_id[subject] = count
                subj_id = count
                count += 1
            else:
                subj_id = str_to_id[subject]

            if property not in tables:
                tables[property] = []
            tables[property].append({"object": obj_id, "subject": subj_id})
            # tables[property].append((object, subject.strip()))

    return tables

tables_id = get_triplets()

In [8]:
#Get triplets from file watdiv.10M.nt and create a table for each property
# and create a dictionary of index/ string
str_to_id = {}
def get_triplets():
    tables = {}
    with open("watdiv.10M.nt", "r") as f:
        count = 0
        for line in f:
            object, property, subject = line.split("\t")
            object = object.split(">")[0].split("/")[-1]
            subject = subject.split(">")[0].split("/")[-1]
            property = property.split(">")[0].split("/")[-1]
            subject = subject.strip()
            if object not in str_to_id:
                str_to_id[object] = count
                obj_id = count
                count += 1
            else:
                obj_id = str_to_id[object]
                
            if subject not in str_to_id:
                str_to_id[subject] = count
                subj_id = count
                count += 1
            else:
                subj_id = str_to_id[subject]


            if "#" in property:
                property = property.split("#")[-1]
            
            if property not in tables:
                tables[property] = []
            tables[property].append({"object": obj_id, "subject": subj_id})
    return tables

tables_id = get_triplets()

In [54]:
# this function uses almost the same code as sothMergeJoin function
# difference is it uses ids instead of strings for the join columns
# also experiments shown in place sort is faster than using sorted function in pyhton
# so replaced sorted function with sort function
# this function is faster than the other one
def sortMergeJoin_1(table1, index1, table2, index2, historyName):
    def merge(s, r):
        i, j = 0, 0
        while i < len(s) and j < len(r):

            key1 = s[i][index1]
            key2 = r[j][index2]

            if key1 == key2:
                joined_item = {**s[i], **r[j]}

                if "history" not in joined_item:
                        joined_item["history"] = set()
                joined_item["history"].add((s[i]["object"], s[i]["subject"], historyName))
                
                yield joined_item
                j += 1
            elif key1 < key2:
                i += 1
            else:
                j += 1
        return 

    # Sort both tables based on the join columns
    table1.sort(key=lambda x: x[index1])
    table2.sort(key=lambda x: x[index2])

    return list(merge(table1, table2))


from time import time
start_time = time()

res = sortMergeJoin_1(tables_id["follows"], "object", tables_id["friendOf"], "subject", "follows")
# show(res, 20)

res = sortMergeJoin_1(res, "subject", tables_id["likes"], "object", "friendOf")
# show(res, 20)

res = sortMergeJoin_1(res, "subject", tables_id["hasReview"], "object", "likes")
# show(res, 20)

print("Program execution time is:", str(time()-start_time))



Program execution time is: 19.96567988395691


In [56]:

# convert id s back to strings

list_of_keys = list(str_to_id.keys())
for item in res:
    history_list = list(item['history'])
    new_history_list = []
    for group in history_list:
        group = (list_of_keys[group[0]], list_of_keys[group[1]], group[2])
        new_history_list.append(group)
    # replace old dictionary with new one
    item['history'] = set(new_history_list)


# wirteToFileAsTable(res, output="output-sortMergeJoin-10M-table.txt")
wirteToFileAsTable(res, output="output-sortMergeJoin-10M-opt_table.txt")

Writing to file...


In [ ]:
# As it can be seen using indexes instead of strings is faster 24s vs 19s for the big database